In [2]:
# %pip install openai python-dotenv pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 2.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.5 MB 2.9 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.5 MB 2.8 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.5 MB 2.7 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.5 MB 2.7 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.5 MB 1.8 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.5 MB 1.8 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.5 MB 1.7 MB/s eta 0:00:06
   ---------- ----------------------------- 2.9/11.5 MB 1.6 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/11.5 MB 1.5 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/11.5 MB 1.5 MB/s eta 0:00:06
   ----------- ---------------------------- 3.4/11.5 MB 1.4 MB/s eta 0:00:06
   ---


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import rapidjson
import pandas as pd
from os import getenv

In [11]:
load_dotenv()

False

In [12]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  # api_key=getenv("OPENROUTER_API_KEY"),
  api_key="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",
)

In [24]:
def generate_mcqs(context, assessment_los, num_questions=30):
    """
    Generate MCQs from a given context using an LLM.
    """
    prompt = f"""
    You are a master educator that is proficient in teaching students and 
    creatting multiple-choice questions (MCQs) assessment that assesses their 
    understanding of given topics and achieves the learning outcomes of the assessment.
    Generate {num_questions} MCQs based on the following context which includes a list of topics covered in a Python programming course, 
    and the learning outcomes of the assessment.
    Each question should have 4 options, with one correct answer and three distractors.
    Please ensure the questions are relevant, clear, and cover key aspects of the context.
    The questions should be in a variety of difficulty levels (i.e. easy, medium, hard).
    
    Context: {context}
    Assessment learning outcomes: {assessment_los}

    Required JSON format:
        {{
            "question": "<question text>",
            "options": {{
                "A": "<option 1>",
                "B": "<option 2>",
                "C": "<option 3>",
                "D": "<option 4>",
            }},
            "correct_answer": "<Correct Option letter"            
        }}
    
    Please return only the generated MCQs as a json
    
    """
    
    response = client.chat.completions.create(
        model="google/gemini-2.0-flash-lite-preview-02-05:free",
        response_format={ "type": "json_object" },        
        messages=[
            {"role": "system", "content": "Always respond with valid JSON in the exact format specified. The json needs to start with [{\"question\":"},            
            {"role": "user", "content": prompt}
        ],
        # max_tokens=1000
    )
    
    mcqs = response.choices[0].message.content
    return mcqs

In [29]:
# Read the context from a text file 
context = ""
with open("content.txt", "r") as f:
    context = f.read()
    
print(context)

Topic 1: Introduction to Problem Solving, Programming and Software Dev Methodologies
What is Problem Solving?
â€¢"Finding a solution to a difficult or complex question or situation; to engage in the actions or thoughts necessary to discover solutions to problems."

Types of Problems
â€¢There are lots of different types of problems we can solve -and the difficulty of each problem can change dramatically. For example, let's try solving some of these problemsâ€¦
If I have $50 and purchase 3 boxes of chocolates on sale for $9 each, how much change will I receive?
How can we reverse climate change?
How can we ensure we get good results for all the courses we are studying?

Routine and Non-Routing Problems
â€¢Routine (simple)problems are the kinds of problems have simple solutions and that occur regularly.
â€¢Non-Routine(complicated &complex)problems are more difficult and often require devisingstrategies to solve.
â€¢Then there are wicked problemsâ€¦

Characteristics of Wicked Problems
â€¢D

In [30]:
# Read the learning outcomes from a text file 
assessment_los = ""
with open("A1_los.txt", "r") as f:
    assessment_los = f.read()
    
print(assessment_los)

Knowledge
K1. Identify and use the correct syntax of a common programming language.
K2. Recall and use typical programming constructs to design and implement simple software solutions.
K3. Reproduce and adapt commonly used basic algorithms.
K4. Explain the importance of programming style concepts (documentation, mnemonic names, indentation).
Skills
S1. Utilise pseudocode and/or algorithms as a major program design technique.
S2. Write and implement a solution algorithm using basic programming constructs.



In [31]:
mcqs = generate_mcqs(context, assessment_los)

In [32]:
mcqs = mcqs.split("\n\n")
mcqs

['[\n  {\n    "question": "What is the primary goal of problem-solving?",\n    "options": {\n      "A": "To find a solution to a difficult or complex question or situation.",\n      "B": "To create more problems.",\n      "C": "To ignore complex situations.",\n      "D": "To follow instructions without questioning."\n    },\n    "correct_answer": "A"\n  },\n  {\n    "question": "Which of the following is NOT a characteristic of a \'wicked\' problem?",\n    "options": {\n      "A": "Clear solution",\n      "B": "Difficult to define",\n      "C": "Multiple interdependencies",\n      "D": "Socially complex"\n    },\n    "correct_answer": "A"\n  },\n  {\n    "question": "What is the first step in the Six-Step Problem-Solving Process?",\n    "options": {\n      "A": "Analyze the problem",\n      "B": "Implement the solution",\n      "C": "Identify the problem",\n      "D": "Generate potential solutions"\n    },\n    "correct_answer": "C"\n  },\n  {\n    "question": "Which of the following i

In [36]:


#if type(a_list) == list:
#rapidjson.loads(mcq_list, parse_mode=rapidjson.PM_TRAILING_COMMAS)
print(type(mcq_list)) 

question_list = []

for question_data in mcq_list:
    question_list.append({                
        'Question': question_data['question'],
        'Option A': question_data['options']['A'],
        'Option B': question_data['options']['B'],
        'Option C': question_data['options']['C'],
        'Option D': question_data['options']['D'],
        'Correct Answer': question_data['correct_answer'],
    })
    
df = pd.DataFrame(question_list)
print("--- CSV Questions -------")
print(df.to_csv())
print("----------")


<class 'list'>
--- CSV Questions -------
,Question,Option A,Option B,Option C,Option D,Correct Answer
0,Q1. Which of the following best defines problem-solving?,Memorizing facts and figures.,Finding a solution to a difficult or complex question or situation.,Following instructions without critical thinking.,Avoiding challenges and seeking easy tasks.,B
1,Q2. What is the primary characteristic of a 'routine problem'?,It's highly complex and requires innovative solutions.,It has simple solutions and occurs regularly.,It has no clear solution.,It is difficult to define.,B
2,Q3. Which of the following is NOT a characteristic of a 'wicked problem'?,Difficult to define.,Has many clear dependencies.,Resolution attempts often result in unforeseen consequences.,No clear solution.,B
3,"Q4. According to the text, what is the most important reason for being a good problem solver?",To impress your colleagues.,To get promoted.,"To anticipate, identify, prevent, and solve problems within your area of

In [8]:
def evaluate_mcqs(mcqs, context):
    results = []
    for question in mcqs:
        prompt = f"""
        Evaluate the following MCQ based on the provided context. Evaluate using the provided Rubric.
        Return your evaluation in valid JSON format, without trailing commas. Return with scores (0-1) and justifications.

        Context: {context}

        MCQ: {question}

        Rubric: MCQ Question Evaluation
        Marking Scale: 0.0 (Lowest) - 1.0 (Highest)
        Criteria:
        1) Distractor Effectiveness :
        Assesses the plausibility and relevance of incorrect answer choices (distractors).
        0.0 - 0.2: Distractors are illogical, easily eliminated, or completely unrelated to the question.
        0.3 - 0.5: Distractors are somewhat relevant but easily identified as incorrect by someone with basic knowledge.
        0.6 - 0.8: Distractors are plausible and require some thought to eliminate, but may still have some flaws.
        0.9 - 1.0: Distractors are highly plausible and require a strong understanding of the subject to differentiate from the correct answer.
        2) Guessability:
        Evaluates how easily the correct answer can be guessed without content knowledge.
        0.0 - 0.2: The correct answer is easily guessed through common sense or test-taking strategies.
        0.3 - 0.5: The correct answer might be guessed with some probability, but requires some luck.
        0.6 - 0.8: The correct answer is difficult to guess without content knowledge.
        0.9 - 1.0: The correct answer is virtually impossible to guess without a solid understanding of the subject.
        3) Valid Answer :
        Determines if the question and provided answer are vaild.
        0.0 - 0.0: The provided correct_answer is incorrect. There are 2 or more correct answers. The question is vague or ambiguous.
        1.0 - 1.0: The provided correct_answer is correct, clear and there is only 1 correct answer.
        4) Cognitive Complexity:
        Assesses the level of cognitive processing required to answer the question.
        0.0 - 0.2: Requires simple recall or basic recognition. (Memorization)
        0.3 - 0.5: Requires basic understanding and interpretation. (Foundation)
        0.6 - 0.8: Requires the application of knowledge to new situations or problem-solving. (Application)
        0.9 - 1.0: Requires higher-order thinking, analysis, evaluation, or synthesis.
        5) Discrimination Index:
        Evaluates the question's ability to differentiate between high-performing and low-performing students.
        0.0 - 0.2: High and low-performing students perform similarly; the question does not discriminate.
        0.3 - 0.5: The question shows some discrimination, but there is significant overlap in performance between groups.
        0.6 - 0.8: The question effectively discriminates; high-performing students are more likely to answer correctly, while low-performing students are more likely to answer incorrectly.
        0.9 - 1.0: The question demonstrates excellent discrimination; almost all high-performing students answer correctly, and almost all low-performing students answer incorrectly.

        Required JSON format:
        {{
            "question": "<question_text>",
            "evaluations": {{
                "Distractor_Effectiveness": <0-1>,
                "Distractor_Effectiveness_justification": "<text>",
                "Guessability": <0-1>,
                "Guessability_justification": "<text>",
                "Valid_Answer": <0-1>,
                "Valid_Answer_justification": "<text>",
                "Cognitive_Complexity": <0-1>,
                "Cognitive_Complexity_justification": "<text>",
                "Discrimination_Index": <0-1>,
                "Discrimination_Index_justification": "<text>"
            }}
        }}
        """

        ignore = f"""
                "relevance_score": <0-1>,
                "relevance_justification": "<text>",
                "clarity_score": <0-1>,
                "clarity_justification": "<text>",
                "difficulty_score": <0-1>,
                "difficulty_justification": "<text>",
                "correctness_score": <0-1>,
                "correctness_justification": "<text>"
        """

        try:
            response = client.chat.completions.create(
                model="google/gemini-2.0-flash-lite-preview-02-05:free",
                response_format={ "type": "json_object" },
                messages=[
                    {"role": "system", "content": "You are an MCQ evaluation assistant. Always respond with valid JSON in the exact format specified. The json needs to start with [{\"question\":"},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000
            )
            print("Response:",response)
            print("-----")            
            #print(response.choices)
            #print(response.choices[0])
            #print(response.choices[0].message)
            #print(response.choices[0].message.content)
            eval_data = rapidjson.loads(response.choices[0].message.content, parse_mode=rapidjson.PM_TRAILING_COMMAS)[0]
            results.append({
                'Question': eval_data['question'],
                'Distractor Effectiveness': eval_data['evaluations']['Distractor_Effectiveness'],
                'Distractor Justification': eval_data['evaluations']['Distractor_Effectiveness_justification'],
                'Guessability': eval_data['evaluations']['Guessability'],
                'Guessability Justification': eval_data['evaluations']['Guessability_justification'],
                'Valid Answer': eval_data['evaluations']['Valid_Answer'],
                'Valid Answer Justification': eval_data['evaluations']['Valid_Answer_justification'],
                'Cognitive Complexity': eval_data['evaluations']['Cognitive_Complexity'],
                'Cognitive Complexity Justification': eval_data['evaluations']['Cognitive_Complexity_justification'],
                'Discrimination Index': eval_data['evaluations']['Discrimination_Index'],
                'Discrimination Index Justification': eval_data['evaluations']['Discrimination_Index_justification']

                
                #'Relevance Score': eval_data['evaluations']['relevance_score'],
                #'Relevance Justification': eval_data['evaluations']['relevance_justification'],
                #'Clarity Score': eval_data['evaluations']['clarity_score'],
                #'Clarity Justification': eval_data['evaluations']['clarity_justification'],
                #'Difficulty Score': eval_data['evaluations']['difficulty_score'],
                #'Difficulty Justification': eval_data['evaluations']['difficulty_justification'],
                #'Correctness Score': eval_data['evaluations']['correctness_score'],
                #'Correctness Justification': eval_data['evaluations']['correctness_justification']
                #'Relevance Score': eval_data['evaluations']['relevance_score'],
            })
        except rapidjson.JSONDecodeError as e:
            print("---")
            print(f"Error parsing JSON for question: {question}\nError: {str(e)}")
            print("---")
            #break
            continue
    df = pd.DataFrame(results)
    print("--- CSV -------")
    print(df.to_csv())
    print("--- END CSV -------")
    return df

In [9]:
evaluate_mcqs(mcq_list, context)
# evaluate_mcqs(mcqs[1], context)

Response: ChatCompletion(id='gen-1740543128-dn9pI6QdxUOotcDP6cWH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='[\n  {\n    "question": "Q1. Which of the following best defines problem-solving?",\n    "evaluations": {\n      "Distractor_Effectiveness": 0.8,\n      "Distractor_Effectiveness_justification": "Distractors A, C, and D are somewhat plausible, they represent actions that are related with problem-solving but do not represent the core, meaning they require some thought to eliminate.",\n      "Guessability": 0.6,\n      "Guessability_justification": "The correct answer requires knowing what problem-solving is. One would need content knowledge to get to the correct answer.",\n      "Valid_Answer": 1.0,\n      "Valid_Answer_justification": "The correct answer provided is correct, B: \\"Finding a solution to a difficult or complex question or situation\\" is the definition of problem-solving.",\n      "Cognitive_Complexity": 0

,Question,Distractor Effectiveness,Distractor Justification,Guessability,Guessability Justification,Valid Answer,Valid Answer Justification,Cognitive Complexity,Cognitive Complexity Justification,Discrimination Index,Discrimination Index Justification
0,Q1. Which of the following best defines proble...,0.8,"Distractors A, C, and D are somewhat plausible...",0.6,The correct answer requires knowing what probl...,1.0,"The correct answer provided is correct, B: ""Fi...",0.6,The question requires an understanding of the ...,0.7,The question effectively discriminates; high-p...
1,Q2. What is the primary characteristic of a 'r...,0.7,"Distractors A, C and D are plausible but can b...",0.6,The correct answer requires understanding of t...,1.0,The correct answer is correct.,0.3,Requires basic understanding and recalling the...,0.7,The question effectively differentiates unders...
2,Q3. Which of the following is NOT a characteri...,0.8,Distractor B is plausible; the context mention...,0.7,The question requires understanding the charac...,1.0,The provided correct answer is correct. The qu...,0.7,The question requires application of knowledge...,0.7,This question effectively discriminates. Stude...
3,"Q4. According to the text, what is the most im...",0.7,"Distractors A, B, and D are plausible but not ...",0.6,While some test-takers might eliminate the lea...,1.0,The provided correct answer directly aligns wi...,0.5,The question requires understanding the explic...,0.7,The question is designed to show that good stu...
4,Q5. What is the first step in the six-step pro...,0.7,The distractors are plausible as they represen...,0.7,The correct answer is not easily guessed witho...,1.0,The correct answer is present and is the only ...,0.3,The question requires remembering the first st...,0.7,The question should discriminate well. Student...
5,Q6. Which problem-solving strategy involves tr...,0.7,Distractors A and C are moderately plausible a...,0.7,"The correct answer, trial and error, is less e...",1.0,The correct answer is 'B' and is the only corr...,0.5,The question requires basic understanding of d...,0.7,The question should effectively discriminate a...
6,Q7. Which of the following is NOT a broad cate...,0.8,All the distractors are relevant to computer s...,0.7,The question requires some knowledge of the di...,1.0,The correct answer is valid and there is only ...,0.6,This question requires basic knowledge of prog...,0.7,The question should effectively discriminate b...
7,Q8. Which of the following is an example of an...,0.7,Prolog and Haskell are functional and logic pr...,0.7,A student would need to understand the differe...,1.0,"The correct answer is Python, and only Python ...",0.6,The question requires applying knowledge of di...,0.7,The question should effectively discriminate b...
8,Q9. Which software development methodology emp...,0.7,The distractors are plausible as they represen...,0.6,"The correct answer, Agile development, is not ...",1.0,"The provided correct answer (B) is correct, an...",0.6,The question requires the application of knowl...,0.7,The question should effectively discriminate b...
9,Q10. Which of the following is a characteristi...,0.7,"Options A, B, and D are plausible to some exte...",0.6,The question uses technical terms that are not...,1.0,The correct answer is unequivocally C as expla...,0.5,Requires basic understanding of programming la...,0.7,A basic understanding is required to answer co...


In [ ]:
xmcq_list = [
    {
        "question": "Q1. Which of the following best defines problem-solving?",
        "options": {
            "A": "Memorizing facts and figures.",
            "B": "Finding a solution to a difficult or complex question or situation.",
            "C": "Following instructions without critical thinking.",
            "D": "Avoiding challenges and seeking easy tasks."
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q2. What is the primary characteristic of a 'routine problem'?",
        "options": {
            "A": "It's highly complex and requires innovative solutions.",
            "B": "It has simple solutions and occurs regularly.",
            "C": "It has no clear solution.",
            "D": "It is difficult to define."
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q3. Which of the following is NOT a characteristic of a 'wicked problem'?",
        "options": {
            "A": "Difficult to define.",
            "B": "Has many clear dependencies.",
            "C": "Resolution attempts often result in unforeseen consequences.",
            "D": "No clear solution."
        },
        "correct_answer": "B",
        "difficulty": "Medium"
    },
    {
        "question": "Q4. According to the text, what is the most important reason for being a good problem solver?",
        "options": {
            "A": "To impress your colleagues.",
            "B": "To get promoted.",
            "C": "To anticipate, identify, prevent, and solve problems within your area of expertise in any job.",
            "D": "To avoid making mistakes."
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q5. What is the first step in the six-step problem-solving process?",
        "options": {
            "A": "Implement the Solution",
            "B": "Analyze the problem",
            "C": "Identify the problem",
            "D": "Evaluate the solution"
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q6. Which problem-solving strategy involves trying different approaches until a solution is found?",
        "options": {
            "A": "Using an algorithm",
            "B": "Trial and error",
            "C": "Using a heuristic",
            "D": "Identifying the problem"
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q7. Which of the following is NOT a broad category in Programming Domains?",
        "options": {
            "A": "Scientific Applications",
            "B": "Business Applications",
            "C": "User Interface Design",
            "D": "Artificial Intelligence"
        },
        "correct_answer": "C",
        "difficulty": "Medium"
    },
      {
        "question": "Q8. Which of the following is an example of an Imperative programming language?",
        "options": {
            "A": "Prolog",
            "B": "Lisp",
            "C": "Python",
            "D": "Haskell"
        },
        "correct_answer": "C",
        "difficulty": "Medium"
    },
    {
        "question": "Q9. Which software development methodology emphasizes iterative development and collaboration?",
        "options": {
            "A": "Waterfall model",
            "B": "Agile development",
            "C": "Scrum",
            "D": "None of the above"
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q10. Which of the following is a characteristic of Python?",
        "options": {
            "A": "It is a compiled language.",
            "B": "It is a machine language.",
            "C": "It is an imperative, general-purpose language.",
            "D": "It is primarily used for hardware programming."
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
     {
        "question": "Q11. Which of the following is the recommended approach for creating your first 'Hello World' Python program?",
        "options": {
            "A": "Write the Hello World program first because it is simple program to write and also verifies the development enviroment set up is working as exepected.",
            "B": "Write the Hello World program because it helps to test the computer.",
            "C": "Write the Hello World program after being familiar with other codes.",
            "D": "Write the Hello World program only when necessary."
        },
        "correct_answer": "A",
        "difficulty": "Easy"
    },
    {
        "question": "Q12. What is the correct way to print 'Hello, World!' in Python?",
        "options": {
            "A": "print 'Hello, World!'",
            "B": "System.out.println('Hello, World!')",
            "C": "console.log('Hello, World!')",
            "D": "print(\"Hello, World!\")"
        },
        "correct_answer": "D",
        "difficulty": "Easy"
    },
    {
        "question": "Q13. What is a variable in Python?",
        "options": {
            "A": "A keyword.",
            "B": "A named piece of computer memory.",
            "C": "A function.",
            "D": "A type of data structure."
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q14. Which data type represents Boolean values in Python?",
        "options": {
            "A": "int",
            "B": "str",
            "C": "bool",
            "D": "float"
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q15. How is a string literal defined in Python?",
        "options": {
            "A": "Enclosed in parentheses.",
            "B": "Enclosed in square brackets.",
            "C": "Enclosed in single, double, or triple quotes.",
            "D": "Preceded by the 'str' keyword."
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q16. What will be the output of \"2.2\" + \"Bob\"?",
        "options": {
            "A": "5.  5",
            "B": "TypeError: unsupported operand type(s) for +: 'float' and 'str'",
            "C": "2.2Bob",
            "D": "Error"
        },
        "correct_answer": "B",
        "difficulty": "Medium"
    },
    {
        "question": "Q17. What is the result of 2 + 10 * 5 in Python?",
        "options": {
            "A": "60",
            "B": "70",
            "C": "52",
            "D": "50"
        },
        "correct_answer": "C",
        "difficulty": "Medium"
    },
    {
        "question": "Q18. What is the output of the code: `first = 2; second = 4; result = first / second; print(result)`?",
        "options": {
            "A": "0",
            "B": "0.5",
            "C": "2",
            "D": "2.0"
        },
        "correct_answer": "B",
        "difficulty": "Medium"
    },
    {
        "question": "Q19. Which operator performs integer division in Python 3?",
        "options": {
            "A": "/",
            "B": "//",
            "C": "%",
            "D": "*"
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q20. What will `input()` function return?",
        "options": {
            "A": "int",
            "B": "float",
            "C": "string",
            "D": "boolean"
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q21. What is a conditional statement used for in programming?",
        "options": {
            "A": "Performing a set of instructions repeatedly.",
            "B": "Making decisions in code based on certain conditions.",
            "C": "Storing multiple values in a single variable.",
            "D": "Defining a function."
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q22. How does Python determine code blocks within 'if' statements and loops?",
        "options": {
            "A": "Using curly braces {}.",
            "B": "With the 'begin' and 'end' keywords.",
            "C": "By indentation.",
            "D": "Using semicolons."
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q23. What is the purpose of the 'elif' statement?",
        "options": {
            "A": "To define a new loop.",
            "B": "To create a single-line 'if' statement.",
            "C": "To handle multiple conditions in an 'if' statement.",
            "D": "To terminate a conditional statement."
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q24. What does `myNumber+= 10` do?",
        "options": {
            "A": "Adds 10 to myNumber.",
            "B": "Subtracts 10 from myNumber.",
            "C": "Multiplies myNumber by 10.",
            "D": "Assigns the value of 10 to myNumber."
        },
        "correct_answer": "A",
        "difficulty": "Easy"
    },
    {
        "question": "Q25. In which scenario will the following code print 'Hmm, I feel like having a cold drink'?",
        "options": {
            "A": "thirsty = False; tooHot = True; if (thirsty == True) or (tooHot== True)",
            "B": "thirsty = True; tooHot = False; if (thirsty == True) or (tooHot== True)",
            "C": "thirsty = False; tooHot = False; if (thirsty == True) or (tooHot== True)",
            "D": "thirsty = False; tooHot = True; if (thirsty == False) or (tooHot== False)"
        },
        "correct_answer": "B",
        "difficulty": "Medium"
    },
    {
        "question": "Q26. What type of data structure is a list in Python?",
        "options": {
            "A": "A fixed-size array.",
            "B": "A dynamic structure that can store different data types.",
            "C": "A variable that can only hold a single value.",
            "D": "An immutable sequence."
        },
        "correct_answer": "B",
        "difficulty": "Easy"
    },
    {
        "question": "Q27. What is the value of `beatles[4]` for `beatles = [\"John\", \"Paul\", \"George\", \"Ringo\"]`?",
        "options": {
            "A": "\"John\"",
            "B": "\"Ringo\"",
            "C": "Error: list index out of range",
            "D": "None"
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q28. What is the output of `print(list(range(4, 10)))`?",
        "options": {
            "A": "[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",
            "B": "[0, 1, 2, 3, 4]",
            "C": "[4, 5, 6, 7, 8, 9]",
            "D": "[4, 5, 6, 7, 8, 9, 10]"
        },
        "correct_answer": "C",
        "difficulty": "Easy"
    },
    {
        "question": "Q29. Which function can be used extract a substring?",
        "options": {
            "A": "split()",
            "B": "replace()",
            "C": "find()",
            "D": "isalnum()"
        },
        "correct_answer": "C",
        "difficulty": "Medium"
    },
{
        "question": "Q30. Which of the following statement about Python file operations is FALSE?",
        "options": {
            "A": "We can write string data to files.",
            "B": "There are only two types of files that can be handled in Python: text files and binary files.",
            "C": "The data that is written to file needs to be converted to string format first.",
            "D": "The 'w' access mode can Truncate and Overwrite existing files."
        },
        "correct_answer": "A",
        "difficulty": "Medium"
    }
]
